# Step 1: Crop People Images

Reference: https://medium.com/@Mert.A/how-to-segment-objects-with-yolov9-71a3439c8b10

## Installing the necessary libraries

In [ ]:
!uv sync

## Importing libraries

In [1]:
import os
import random
import numpy as np
import cv2
from ultralytics import YOLO

## Choose your model

In [2]:
model = YOLO("../weights/yolov9e-seg.pt")

## Segmenting Objects in Images with YOLOv9

Settings

In [3]:
source_path = "../dataset/people"
save_path = "../dataset/person"
save_file_num = 1

classes_ids = [0]  # person
conf = 0.2

Process

In [ ]:
image_files = os.listdir(source_path)
for image_file in image_files:
    img = cv2.imread(os.path.join(source_path, image_file))

    results = model.predict(img, conf=conf)
    # colors = [random.choices(range(256), k=3) for _ in classes_ids]
    print(results)

    result = results[0]
    for mask, box in zip(result.masks.xy, result.boxes):
        points = np.int32([mask])
        if int(box.cls[0]) in classes_ids:
            img_copy = img.copy()

            # Draw the mask
            # cv2.polylines(img_copy, points, True, (255, 0, 0), 1)
            # color_number = classes_ids.index(int(box.cls[0]))
            # cv2.fillPoly(img_copy, points, colors[color_number])

            # Crop the image
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cropped_img = img[y1:y2, x1:x2]

            # Create a mask for the cropped image
            mask_img = np.zeros_like(cropped_img)
            mask_points = points - [x1, y1]
            cv2.fillPoly(mask_img, mask_points, (255, 255, 255))

            # Apply the mask to the cropped image
            masked_cropped_img = cv2.bitwise_and(cropped_img, mask_img)

            # show images
            # cv2.imshow("Masked Cropped Image", masked_cropped_img)
            # cv2.imshow("Image", img_copy)
            # cv2.waitKey(0)

            # save image
            cv2.imwrite(os.path.join(save_path, f"image_{save_file_num:06d}.jpg"), masked_cropped_img)
            save_file_num += 1

# Step 2: Label People's Clothes

Use tools like https://www.makesense.ai to label images and export annotations as YOLO format.

If you need to modify hundreds of thousands of annotation files, you can use `src/train_utility.ipynb` to do this.

# Step 3: Train the Model

Reference: https://docs.ultralytics.com/modes/train/#usage-examples

In [1]:
from ultralytics import YOLO

To train on multi-GPU or Apple Silicon, please read the reference article.

In [ ]:
model = YOLO("../weights/yolo11s.pt")
results = model.train(data="../dataset/mask/mask.yaml", epochs=16, batch=-1, imgsz=640)

After the training complete, move the `src/runs/detect/train/weights/best.pt` to `weights/clothes.pt`.